# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as st
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
geo_data_0 = pd.read_csv('/datasets/geo_data_0.csv') 
geo_data_1 = pd.read_csv('/datasets/geo_data_1.csv') 
geo_data_2 = pd.read_csv('/datasets/geo_data_2.csv')

In [3]:
for d in [geo_data_0, geo_data_1, geo_data_2]:
    print(d.info(), end='\n\n')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column

Пропусков нет, можем исследовать данные дальше.

In [4]:
for d in [geo_data_0, geo_data_1, geo_data_2]:
    print(d.head(10), end='\n\n')

      id        f0        f1        f2     product
0  txEyH  0.705745 -0.497823  1.221170  105.280062
1  2acmU  1.334711 -0.340164  4.365080   73.037750
2  409Wp  1.022732  0.151990  1.419926   85.265647
3  iJLyR -0.032172  0.139033  2.978566  168.620776
4  Xdl7t  1.988431  0.155413  4.751769  154.036647
5  wX4Hy  0.969570  0.489775 -0.735383   64.741541
6  tL6pL  0.645075  0.530656  1.780266   49.055285
7  BYPU6 -0.400648  0.808337 -5.624670   72.943292
8  j9Oui  0.643105 -0.551583  2.372141  113.356160
9  OLuZU  2.173381  0.563698  9.441852  127.910945

      id         f0         f1        f2     product
0  kBEdx -15.001348  -8.276000 -0.005876    3.179103
1  62mP7  14.272088  -3.475083  0.999183   26.953261
2  vyE1P   6.263187  -5.948386  5.001160  134.766305
3  KcrkZ -13.081196 -11.506057  4.999415  137.945408
4  AHL4O  12.702195  -8.147433  5.004363  134.766305
5  HHckp  -3.327590  -2.205276  3.003647   84.038886
6  h5Ujo -11.142655 -10.133399  4.002382  110.992147
7  muH9x   4.2

Можем заметить, что в датафреймах есть столбец *id*, который не нужен для обучения модели. Поэтому можем его удалить.

In [5]:
geo_data_0 = geo_data_0.drop(['id'], axis=1)
geo_data_1 = geo_data_1.drop(['id'], axis=1)
geo_data_2 = geo_data_2.drop(['id'], axis=1)

In [6]:
for d in [geo_data_0, geo_data_1, geo_data_2]:
    print(d.columns)

Index(['f0', 'f1', 'f2', 'product'], dtype='object')
Index(['f0', 'f1', 'f2', 'product'], dtype='object')
Index(['f0', 'f1', 'f2', 'product'], dtype='object')


**Вывод:** успешно загрузили и проверили данные, удалили ненужный столбец *id*.


## Обучение и проверка модели

In [50]:
# Регион 1
features_0 = geo_data_0.drop(['product'], axis=1)
target_0 = geo_data_0['product']
features_train_0, features_valid_0, target_train_0, target_valid_0 = train_test_split(features_0, target_0, test_size=0.25, random_state=12345)
model_0 = LinearRegression() 
model_0.fit(features_train_0, target_train_0) 
predictions_valid_0 = model_0.predict(features_valid_0)
m_predictions_valid_0 = pd.Series(predictions_valid_0)
m_predictions_valid_0.reset_index(drop=True, inplace=True)
rmse_0 = mean_squared_error(target_valid_0, predictions_valid_0)**0.5

print('Регион 1', '\n')
print('RMSE:', rmse_0)
print('Предсказанный cредний запас сырья:', predictions_valid_0.mean())
print('Истинный cредний запас сырья:', target_valid_0.mean(), '\n\n')


# Регион 2
m_predictions_valid_1 = []
features_1 = geo_data_1.drop(['product'], axis=1)
target_1 = geo_data_1['product']
features_train_1, features_valid_1, target_train_1, target_valid_1 = train_test_split(features_1, target_1, test_size=0.25, random_state=12345)
model_1 = LinearRegression() 
model_1.fit(features_train_1, target_train_1) 
predictions_valid_1 = model_1.predict(features_valid_1)
m_predictions_valid_1 = pd.Series(predictions_valid_1)
m_predictions_valid_1.reset_index(drop=True, inplace=True)
rmse_1 = mean_squared_error(target_valid_1, predictions_valid_1)**0.5

print('Регион 2', '\n')
print('RMSE:', rmse_1)
print('Предсказанный cредний запас сырья:', predictions_valid_1.mean())
print('Истинный cредний запас сырья:', target_valid_1.mean(), '\n\n')


# Регион 3
features_2 = geo_data_2.drop(['product'], axis=1)
target_2 = geo_data_2['product']
features_train_2, features_valid_2, target_train_2, target_valid_2 = train_test_split(features_2, target_2, test_size=0.25, random_state=12345)
model_2 = LinearRegression() 
model_2.fit(features_train_2, target_train_2) 
predictions_valid_2 = model_2.predict(features_valid_2)
m_predictions_valid_2 = pd.Series(predictions_valid_2)
m_predictions_valid_2.reset_index(drop=True, inplace=True)
rmse_2 = mean_squared_error(target_valid_2, predictions_valid_2)**0.5

print('Регион 3', '\n')
print('RMSE:', rmse_2)
print('Предсказанный cредний запас сырья:', predictions_valid_2.mean())
print('Истинный cредний запас сырья:', target_valid_2.mean(), '\n\n')

Регион 1 

RMSE: 37.5794217150813
Предсказанный cредний запас сырья: 92.59256778438035
Истинный cредний запас сырья: 92.07859674082927 


Регион 2 

RMSE: 0.893099286775617
Предсказанный cредний запас сырья: 68.728546895446
Истинный cредний запас сырья: 68.72313602435997 


Регион 3 

RMSE: 40.02970873393434
Предсказанный cредний запас сырья: 94.96504596800489
Истинный cредний запас сырья: 94.88423280885438 




**Вывод:** модель показала наилучший среди остальных результат RMSE равным ~0.89 во втором по счету регионе. Но в этом регионе самый низкий предсказанный средний запас сырья, поэтому можно считать его малоприбыльным отностильно двух других регионов. Что касается предсказнного и истинного значения среднего запаса сырья, то они почти совпадают, что говорит нам о хорошем качестве обученной модели.

## Подготовка к расчёту прибыли

Создадим константы для дальнейших вычислений:

In [18]:
ALL_POINTS = 500
BEST_POINTS = 200                 #200 лучших точек для разработки
BUDGET = 10_000_000_000        #бюджет на разработку скважин в регионе — 10 млрд рублей
INCOME = 450_000

Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины: 

In [22]:
sufficient_volume = BUDGET / (BEST_POINTS * INCOME)

print('Достаточный объём сырья для безубыточной разработки новой скважины:', sufficient_volume)
print('Объем сырья в скважине №1:', geo_data_0['product'].mean())
print('Объем сырья в скважине №2:', geo_data_1['product'].mean())
print('Объем сырья в скважине №3:', geo_data_2['product'].mean())

Достаточный объём сырья для безубыточной разработки новой скважины: 111.11111111111111
Объем сырья в скважине №1: 92.50000000000001
Объем сырья в скважине №2: 68.82500000000002
Объем сырья в скважине №3: 95.00000000000004


**Вывод:** как мы можем заметить, средние показатели объема сырья в каждом регионе заметно ниже, чем показатель достаточного объёма сырья для безубыточной разработки новой скважины.

Теперь напишем функцию по расчету прибыли:

In [52]:
def profit(target, probabilities, revenue, count): 
    probs_sorted = probabilities.sort_values(ascending=False) 
    selected = target[probs_sorted.index][:count]             
    return revenue * selected.sum() - BUDGET

## Расчёт прибыли и рисков 

In [55]:
# predictions_valid_0_s = pd.Series(predictions_valid_0)   
# target_valid_s = pd.Series(target_valid).reset_index(drop=True)  

# state = np.random.RandomState(12345)
    
# values_0 = []
    
# for i in range(1000):
#     target_subsample = target_valid_s.sample(n=500, replace=True, random_state=state) 
#     probs_subsample = predictions_valid_0_s[target_subsample.index]
#     values_0.append(profit(target_subsample, probs_subsample, INCOME, BEST_POINTS))

# values_0 = pd.Series(values_0)

# mean_0 = values_0.mean()
# lower_0 = values_0.quantile(.025)
# upper_0 = values_0.quantile(.975)
# risk_0 = st.percentileofscore(values_0, 0)
    
# print('Регион 0', '\n')
# print('Средняя выручка:', mean_0)
# print('Доверительный интервал: [', lower_0, ':', upper_0, ']')
# print('Риск:', risk_0, '%')




# predictions_valid_1_s = pd.Series(predictions_valid_1) 


# state = np.random.RandomState(12345)
    
# values_1 = []
    
# for i in range(1000):
#     target_subsample = target_valid_s.sample(n=500, replace=True, random_state=state) 
#     probs_subsample = predictions_valid_1_s[target_subsample.index]
#     values_1.append(profit(target_subsample, probs_subsample, INCOME, BEST_POINTS))

# values_1 = pd.Series(values_1)

# mean_1 = values_1.mean()
# lower_1 = values_1.quantile(.025)
# upper_1 = values_1.quantile(.975)
# risk_1 = st.percentileofscore(values_1, 0)
    
# print('Регион 1', '\n')
# print('Средняя выручка:', mean_1)
# print('Доверительный интервал: [', lower_1, ':', upper_1, ']')
# print('Риск:', risk_1, '%')

Регион 0 

Средняя выручка: -1466761335.3934894
Доверительный интервал: [ -2053404437.3309832 : -875805850.7742691 ]
Риск: 100.0 %
Регион 1 

Средняя выручка: -1427684719.833217
Доверительный интервал: [ -2068771572.458009 : -855800998.2685544 ]
Риск: 100.0 %


In [61]:
num = 0
for dataset in [geo_data_0, geo_data_1, geo_data_2]:
    features = dataset.drop(['product'], axis=1)  
    target = dataset['product']  
    features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.25, random_state=12345)
    
    model = LinearRegression()
    model.fit(features_train, target_train) 
    predictions_valid = model.predict(features_valid) 
    
    predictions_valid_ = pd.Series(predictions_valid) 
    target_valid_ = pd.Series(target_valid).reset_index(drop=True)

    state = np.random.RandomState(12345)
    
    values = []
    
    for i in range(1000):
        target_subsample = target_valid_.sample(n=500, replace=True, random_state=state) 
        probs_subsample = predictions_valid_[target_subsample.index]
        values.append(profit(target_subsample, probs_subsample, INCOME, BEST_POINTS))

    values = pd.Series(values)

    lower = values.quantile(.025)
    upper = values.quantile(.975)
    risk = st.percentileofscore(values, 0)
    
    print('Для региона', num)
    print('Средняя выручка:', values.mean())
    print('Доверительный интервал: от ', lower, 'до', upper)
    print('Риск:', risk, '%')
    
    num += 1

Для региона 0
Средняя выручка: 425938526.91059244
Доверительный интервал: от  -102090094.83793654 до 947976353.3583689
Риск: 6.0 %
Для региона 1
Средняя выручка: 515222773.4432899
Доверительный интервал: от  68873225.37050176 до 931547591.2570494
Риск: 1.0 %
Для региона 2
Средняя выручка: 435008362.7827556
Доверительный интервал: от  -128880547.32978901 до 969706954.1802661
Риск: 6.4 %


## Вывод

Первый и третий регионы обладают достаточно высоким процентом риска и имеют отрицательное значение прибыли на левой границе доверительного интервала, следовательно они не подходят под условия нашей задачи. **Второй регион** обладает процентом риска всего 1%, что подходит под условия нашей задачи; также обладает положительной левой границей и самым высоким средним доходом из трех регионов. Следовательно для разработки нового месторождения лучшим регионом из трех будет второй регион.